In [1]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

e:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data\train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 使用更加方便的InteractiveSession类
# 能让你在运行图的时候，插入一些计算图,对于工作在交互式环境IPython中的人们来说非常便利
# 如果你没有使用InteractiveSession
# 那么你需要在启动session之前构建整个计算图，然后启动该计算图。
import tensorflow as tf
sess = tf.InteractiveSession()

# 占位符
# x和y并不是特定的值，相反，他们都只是一个占位符
# 在TensorFlow运行某一计算时根据该占位符输入具体的值。
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

# 变量
# 定义权重W和偏置b
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# 初始化变量
# 为初始值指定具体值（本例当中是全为零），并将其分配给每个变量,
# 可以一次性为所有变量完成此操作
sess.run(tf.global_variables_initializer())

# 类别预测
y = tf.nn.softmax(tf.matmul(x,W) + b)

# 损失函数
# 损失函数是目标类别和预测类别之间的交叉熵
cross_entropy = - tf.reduce_sum(y_*tf.log(y))

# 训练模型
# 我们用最速下降法让交叉熵下降，步长为0.01.
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# 整个模型的训练可以通过反复地运行train_step来完成
for i in range(1000):
  batch = mnist.train.next_batch(50)
  # 通过feed_dict将x 和 y_张量占位符用训练训练数据替代。
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

#评估模型
# 最大值1所在的索引位置就是类别标签
# 用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# 将布尔值转换为浮点数来代表对、错，然后取平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# 测试数据上的准确率
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9135


In [3]:
# 构建一个多层卷积网络

# 权重初始化
# 权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
# 使用的是ReLU神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，
# 以避免神经元节点输出恒为0的问题（dead neurons）
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
# 我们的卷积使用1步长（stride size），0边距（padding size）的模板，
# 保证输出和输入是同一个大小
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# 池化用简单传统的2x2大小的模板做max pooling
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [4]:
# 第一层卷积 : 一个卷积接一个max pooling完成

# 权重张量形状是[5, 5, 1, 32]，前两个维度是patch的大小，
# 接着是输入的通道数目，最后是输出的通道数目
W_conv1 = weight_variable([5, 5, 1, 32])
# 每一个输出通道都有一个对应的偏置量
b_conv1 = bias_variable([32])

#为了用这一层，我们把x变成一个4d向量，
# 其第2、第3维对应图片的宽、高，最后一维代表图片的颜色通道数
# (因为是灰度图所以这里的通道数为1，如果是rgb彩色图，则为3)。
x_image = tf.reshape(x, [-1,28,28,1])

# x_image和权值向量进行卷积，加上偏置项，然后应用ReLU激活函数，
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 最后进行max pooling
h_pool1 = max_pool_2x2(h_conv1)

In [5]:
# 第二层卷积
# 每个5x5的patch会得到64个特征
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [6]:
# 密集连接层
# 图片尺寸减小到7x7，我们加入一个有1024个神经元的全连接层，用于处理整个图片

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
# 池化层输出的张量reshape成一些向量，乘上权重矩阵，加上偏置，然后对其使用ReLU
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [7]:
# Dropout
# 为了减少过拟合，我们在输出层之前加入dropout

# 注意：：在训练过程中启用dropout，在测试过程中关闭dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层
# 添加一个softmax层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [8]:
# 损失函数
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
# 训练方法
# ADAM优化器来做梯度最速下降
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# 初始化变量
sess.run(tf.global_variables_initializer())

for i in range(1001):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict=
        {x:batch[0], y_: batch[1], keep_prob: 1.0})
    print ("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# 测试的时候，不用dropout
print("test accuracy %g"%accuracy.eval(feed_dict=
    {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))  


step 0, training accuracy 0.06


step 100, training accuracy 0.84


step 200, training accuracy 0.9


step 300, training accuracy 0.82


step 400, training accuracy 0.98


...

step 600, training accuracy 0.94


step 700, training accuracy 0.96


step 800, training accuracy 0.98


step 900, training accuracy 0.96


step 1000, training accuracy 0.94
